In [ ]:
import numpy as np
import os

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from pyproj import Proj
import xarray as xr
from itertools import chain

def line_to_float(line):
    data=[]
    for x in line.split(' '):
        if '.' in x:
            data.append(float(x))
    return data    

# main script
log_file = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet/test_uniform_debug/00810grids/tmp/log2'
root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet/scripts'
ofolder = 'step16_debug_nearet_stn_character'
if not os.path.exists(os.path.join(root_dir,ofolder)):
    os.makedirs(os.path.join(root_dir,ofolder))
ofolder_subs = ['invalid','valid']
for ofolder_sub in ofolder_subs:
    if not os.path.exists(os.path.join(root_dir,ofolder,ofolder_sub)):
        os.makedirs(os.path.join(root_dir,ofolder,ofolder_sub))    
  
print('--save nearest station info---')
# read useful grid lines (grid_id, start_line_num)
records = np.loadtxt(os.path.join(root_dir,ofolder,'GridID_StartLine.txt'),delimiter=',',
                     dtype='int',skiprows=1)
            
# loop through all grids and plot its neighbours
grids_loc = []
for i in range(len(records)-1):
# for i in range(2):
    grid_id = records[i][0]
    grid_start_line = records[i][1]
    grid_end_line = records[i+1][1]
    print('grid_id '+str(grid_id))
    
    # ----read useful metrix----
#     print('(1) read useful matrix')
    with open(log_file) as f:
        for i, line in enumerate(f):
            line=line.strip()
            if line and i>grid_start_line and i<grid_end_line:
                if 'close_meta(3, g, i)=' in line:
                    grid_lat = float(line.split('=')[1])
                if 'close_meta(4, g, i)=' in line:
                    grid_lon = float(line.split('=')[1])
                if 'tx_red_2 =' in line:
                    tx_red_2 = np.asarray(line_to_float(line.split('=')[1]))
                if 'w_pcp_red =' in line:
                    w_pcp_red = np.asarray(line_to_float(line.split('=')[1]))
                if 'y_red =' in line:
                    y_red = np.asarray(line_to_float(line.split('=')[1]))                
                if 'b =' in line:
                    b = np.asarray(line_to_float(line.split('=')[1]))
            elif i>grid_end_line:
                break
    
    if all(b==0.0):
        label = 1
        ofolder_sub = 'invalid'
        print('*** Invalid Grid ***')
    else:
        label = 0
        ofolder_sub = 'valid'
#     grids_loc.append([grid_id,grid_lat,grid_lon,label])

    ny = 224
    nx = 464
    grid_row = grid_id//nx
    grid_col = grid_id%nx

    grid_file = str(grid_id)+'_grid.txt'
    data = (np.asarray([grid_id,grid_lat,grid_lon,label,grid_row,grid_col])).reshape(1,6)
    np.savetxt(os.path.join(root_dir,ofolder, ofolder_sub, grid_file),
               data,fmt='%d,%.4f,%.4f,%d,%d,%d', header='GridId,Latitude,Longitude,Problem(1:Y.0:N),Row(224),Column(464)')
    
    # ----save nearest stn to txt----
#     print('(2) save nearest stn to txt')
    attr_num = 6
    attr_num_target = 4
    stn_num = 30

    x_red=(np.asarray(tx_red_2)).reshape(stn_num,attr_num_target)
    w_pcp_red=(np.asarray(w_pcp_red)).reshape(stn_num,stn_num)
    y_red=(np.asarray(y_red)).reshape(stn_num,1)

    tx_red_2 = x_red.T
    ids = (np.arange(1,stn_num+1)).reshape(stn_num,1)
    w_pcp_red_diag = np.asarray([w_pcp_red[j,j] for j in range(stn_num)]).reshape(stn_num,1)

    data_output = np.concatenate((ids,x_red,w_pcp_red_diag,y_red), axis=1)
    x_red_file = str(grid_id)+'_nearest_stn.txt'
    np.savetxt(os.path.join(root_dir,ofolder, ofolder_sub, x_red_file),data_output,
               fmt='%d,%d,%f,%f,%f,%f,%f',header='StnNumber,Unit,latitude,longitude,elevation,weight,p_transfrom')

    # ----singular test----
#     print('(3) singular test')
    twx_red_2 = np.matmul(tx_red_2,w_pcp_red)
    a = np.matmul(twx_red_2,x_red[:,0:attr_num_target])

    a_matrix_ratio_file = str(grid_id)+'_singular_test.txt'
    with open(os.path.join(root_dir,ofolder, ofolder_sub,a_matrix_ratio_file),'w') as f:
        f.write('row/first_row\n')
        for i in range(1,attr_num_target):
            ratio = np.divide(a[i,:],a[0,:])
            for j in range(len(ratio)):
                f.write('%.1f,' % (ratio[j]))
            f.write('\n' % (np.divide(a[i,:],a[0,:])))

#         f.write('\n')
#         f.write('col/first_col\n')
#         for i in range(1,attr_num_target):
#             ratio = np.divide(a[:,i],a[:,0])
#             for j in range(len(ratio)):
#                 f.write('%.2f,' % (ratio[j]))
#             f.write('\n' % (np.divide(a[i,:],a[0,:])))

# # save all grids' locations
# ny = 224
# nx = 464
# grids_loc = np.asarray(grids_loc)
# grids_loc_extend = np.zeros((len(grids_loc),6))
# grids_loc_extend[:,0:4] = grids_loc
# grids_loc_extend[:,4] = grids_loc_extend[:,0]//nx
# grids_loc_extend[:,5] = grids_loc_extend[:,0]%nx
# np.savetxt(os.path.join(root_dir,ofolder,'grids_loc.txt'), grids_loc_extend, 
#            fmt='%d,%.4f,%.4f,%d,%d,%d', header='GridId,Latitude,Longitude,Problem,Row(224),Column(464)')

print('Done')

--save nearest station info---
grid_id 103
grid_id 104
grid_id 105
grid_id 106
grid_id 107
grid_id 108
grid_id 109
grid_id 110
grid_id 111
grid_id 112
grid_id 113
grid_id 114
grid_id 115
grid_id 116
grid_id 134
grid_id 135
grid_id 136
grid_id 137
grid_id 138
grid_id 139
grid_id 140
grid_id 141
grid_id 142
grid_id 143
grid_id 144
grid_id 145
grid_id 146
grid_id 147
grid_id 148
grid_id 149
grid_id 150
grid_id 151
grid_id 152
grid_id 153
grid_id 154
grid_id 155
grid_id 156
grid_id 157
grid_id 158
grid_id 159
grid_id 160
grid_id 161
grid_id 162
grid_id 163
grid_id 164
grid_id 165
grid_id 166
grid_id 167
grid_id 168
grid_id 169
grid_id 170
grid_id 171
grid_id 172
grid_id 173
grid_id 174
grid_id 175
grid_id 176
grid_id 177
grid_id 178
grid_id 179
grid_id 180
grid_id 181
grid_id 182
grid_id 183
grid_id 184
grid_id 185
grid_id 186
grid_id 187
grid_id 188
grid_id 189
grid_id 190
grid_id 191
grid_id 192
grid_id 193
grid_id 194
grid_id 195
grid_id 196
grid_id 197
grid_id 198
grid_id 199
grid_id 2

In [2]:
[grid_id,grid_lat,grid_lon,label,grid_row,grid_col]

[103, 25.0625, -112.1875, 0, 0, 103]